# test de la sincro son, image


dans ce fichier on test la création d'un estimateur tres simple image/son

La personne qui parle est celle la plus proche de la source de son.

In [2]:
import inspect

from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, KFold

from metric.compute_metric import compute_clear
from utils import make_pred, save_data, iter_frame_sync
from metric.read_file import read_file
from utils import Point, mult_load, compare_corp_son
import numpy as np


In [2]:
from utils import iter_frame, Corp
from utils.identity_assignation import identity_assignation


class InertieClasificator(BaseEstimator, ClassifierMixin):
    def __init__(self, coef_ine_hd = 1, coef_ine_bd =1, coef_dist_hd = 1, coef_dist_bd = 1, 
                 coef_size_hd = 1, coef_size_bd = 1, coef_pt_cache = 10, mem_length= 20, coef_col = 1):
        
        args, _, _, values = inspect.getargvalues(inspect.currentframe())
        values.pop("self")

        for arg, val in values.items():
            setattr(self, arg, val)
    
    def fit(self, X, y=None):
        # on n'apprend rien pour l'instant
        pass
    
    def mk_dist(self):
        def fdist(id, corp):
            var = id.comp(corp)
            dist = var[0] * self.coef_ine_hd
            dist += var[1] * self.coef_ine_bd
            dist += var[2] * self.coef_col
            dist += var[3] * self.coef_dist_hd
            dist += var[4] * self.coef_dist_bd
            dist += var[5] * self.coef_pt_cache
            return dist
        return fdist
               
    
    def predict(self, X, y=None):
        # prediction
        
        def func (data):
            data_pos, data_son = data
            identites = dict()
            for frame_pos, frame_son in iter_frame_sync(data_pos,data_son):
                
                son = frame_son[0]
                identites = identity_assignation(frame_pos, identites, None, self.mk_dist())
    
                for i, l in enumerate(frame_pos):
                    cp = Corp(l)
                    parle = 0
                    # print(str(Point(son[2], son[1])) + "vs " + str(cp.hd_bary))
                    if cp.hd_bary.dist(Point(son[2], son[1])) < 50:
                        # print("Match !! ")
                        parle = 1
                    yield (identites[cp], parle)  
        
        return [make_pred(vid[0], func, data_son=vid[1]) for vid in X]
    
    def score(self, X, y, sample_weight=None):
        pred = self.predict(X)
        res = [0, 0, 0, 0]
        for vid, raw in zip(pred, y):
            c = compute_clear(raw, vid, 1)
            c = list(c)
            res = [a+b for a,b in zip(res, c)]
        # print(res/len(y))
        return [x/len(y) for x in res]
           
        

In [38]:
data_ref, data = mult_load(["data/Scenario03-02/", "data/Scenario01-01/", "data/Scenario02-03/", "data/Scenario02-01/",
                            "data/Scenario02-02/", "data/Scenario03-03/", "data/Scenario03-02/", "data/Scenario03-01/",
                            "data/Scenario03-05/"], son=True)

IOError: [Errno 2] No such file or directory: 'data/Scenario03-02/ssl_detection.txt'

In [7]:
est = InertieClasificator()
print(est.score(data, data_ref))

[0.97933572191362606, 0.0, 0.74515192766969995, 0.72656087985525009]


parole a 0 : 0.977; 0 , 0.54, 0.53

seuil 50 > 0.745/0.726
seuil 30/ 40 > 0.71, 0.69
seuil 30/ 70 > pareil
seuil 50/50 > 0.76, 0.747

In [8]:
PATH = "data/Scenario02-01/"
data_ref = read_file(PATH+"groundTruth.txt")
data = np.loadtxt(PATH+"augmented_data.txt")
data_son = np.loadtxt(PATH+"ssl_detections.txt")
est = InertieClasificator()
res = est.predict([(data, data_son)], [data_ref])
print(est.score([(data, data_son)], [data_ref]))

[0.98086419753086418, 0.0, 0.62647607209446865, 0.6098765432098765]


In [9]:
res

[array([[  19.,    5.,    0., ...,   42.,   -1.,   -1.],
        [  20.,    5.,    0., ...,   48.,   -1.,   -1.],
        [  21.,    5.,    0., ...,   62.,   -1.,   -1.],
        ..., 
        [ 848.,    6.,    0., ...,   -1.,   34.,   98.],
        [ 849.,    6.,    0., ...,   -1.,   22.,   98.],
        [ 850.,    6.,    0., ...,   -1.,   -1.,   -1.]])]

In [10]:
save_data(res[0], PATH +  "res_tmp.txt")

In [3]:
from utils import iter_frame, Corp
from utils.identity_assignation import identity_assignation


class InertieClasificator(BaseEstimator, ClassifierMixin):
    def __init__(self, coef_ine_hd = 1, coef_ine_bd =1, coef_dist_hd = 1, coef_dist_bd = 1, 
                 coef_size_hd = 1, coef_size_bd = 1, coef_pt_cache = 10, mem_length= 20, coef_col = 1,
                 seuil_parole = 60, bonus_parole = 10):
        
        args, _, _, values = inspect.getargvalues(inspect.currentframe())
        values.pop("self")
        self.seuil_parole = seuil_parole
        self.bonus_parole = bonus_parole

        for arg, val in values.items():
            setattr(self, arg, val)
    
    def fit(self, X, y=None):
        # on n'apprend rien pour l'instant
        pass
    
    def mk_dist(self):
        def fdist(id, corp):
            var = id.comp(corp)
            dist = var[0] * self.coef_ine_hd
            dist += var[1] * self.coef_ine_bd
            dist += var[2] * self.coef_col
            dist += var[3] * self.coef_dist_hd
            dist += var[4] * self.coef_dist_bd
            dist += var[5] * self.coef_pt_cache
            return dist
        return fdist
               
    
    def predict(self, X, y=None):
        # prediction
        
        def func (data):
            data_pos, data_son = data
            identites = dict()
            for frame_pos, frame_son in iter_frame_sync(data_pos,data_son):
                
          
                identites = identity_assignation(frame_pos, identites, None, self.mk_dist())
    
                for i, l in enumerate(frame_pos):
                    cp = Corp(l)
                    l = [Point(x[2], x[1]) for x in frame_son]
                    pson = min(l, key=lambda p:cp.hd_bary.dist(p))    
                    
                    id = None
                    for _id in identites.keys():
                        if _id == cp:
                            id = _id
                    parle = False
                    # print(str(Point(son[2], son[1])) + "vs " + str(cp.hd_bary))                    
                    if cp.hd_bary.dist(pson) < self.seuil_parole + int(id.get_speak())*self.bonus_parole :
                        # print("Match !! ")
                        parle = True
                    id.set_speak(parle)
                    yield (identites[cp], int(parle))  
        
        return [make_pred(vid[0], func, data_son=vid[1]) for vid in X]
    
    def score(self, X, y, sample_weight=None):
        pred = self.predict(X)
        res = [0, 0, 0, 0]
        for vid, raw in zip(pred, y):
            c = compute_clear(raw, vid, 1)
            c = list(c)
            res = [a+b for a,b in zip(res, c)]
        # print(res/len(y))
        return [x/len(y) for x in res]
           
        

In [5]:
data_ref, data = mult_load(["data/Scenario03-02/", "data/Scenario01-01/", "data/Scenario02-03/", "data/Scenario02-01/",
                            "data/Scenario02-02/", "data/Scenario03-03/", "data/Scenario03-02/","data/Scenario03-01/",
                            "data/Scenario03-05/"], son=True, ficher_son="mixture")
data_ref, data = mult_load(["test/Scenario01-03/", "test/Scenario03-MeetingRoom04/", "test/Scenario05-03/", "test/Scenario05-04/"], son=True, ficher_son="mixture")
for s in [10, 30, 40, 50, 60, 70, 80, 100]:
    for b in [10,20,30,40,50,60]:
        est = InertieClasificator(seuil_parole=s, bonus_parole=b)
        print("res pour s = {} et b = {}".format(s, b))
        print(est.score(data, data_ref))

res pour s = 10 et b = 10


[0.98442361276303036, 0.0, 0.4773838072879486, 0.46475058067484498]
res pour s = 10 et b = 20


[0.98442361276303036, 0.0, 0.49674446267639843, 0.48400435748338061]
res pour s = 10 et b = 30


[0.98442361276303036, 0.0, 0.53869995530217707, 0.52575424187434727]
res pour s = 10 et b = 40


[0.98442361276303036, 0.0, 0.5575601772887504, 0.54454146638922662]
res pour s = 10 et b = 50


[0.98442361276303036, 0.0, 0.56080452163949712, 0.54777254829304978]
res pour s = 10 et b = 60


[0.98442361276303036, 0.0, 0.5636866851902429, 0.55063675974956761]
res pour s = 30 et b = 10


KeyboardInterrupt: 

res pour s = 10 et b = 10


[0.98086419753086418, 0.0, 0.35052827843380979, 0.33580246913580247]
res pour s = 10 et b = 20


[0.98086419753086418, 0.0, 0.3523927905531386, 0.33765432098765435]
res pour s = 10 et b = 30


[0.98086419753086418, 0.0, 0.35550031075201988, 0.34074074074074079]
res pour s = 10 et b = 40


[0.98086419753086418, 0.0, 0.35985083903045367, 0.34506172839506177]
res pour s = 10 et b = 50


...

[0.98086419753086418, 0.0, 0.62647607209446865, 0.6098765432098765]
res pour s = 40 et b = 30


[0.98086419753086418, 0.0, 0.64077066500932256, 0.624074074074074]
res pour s = 40 et b = 40


[0.98086419753086418, 0.0, 0.65692977004350528, 0.6401234567901235]
res pour s = 40 et b = 50


[0.98086419753086418, 0.0, 0.66749533871970168, 0.65061728395061724]
res pour s = 40 et b = 60


KeyboardInterrupt: 

In [8]:
data_ref, data = mult_load(["test/Scenario01-03/", "test/Scenario03-MeetingRoom04/", "test/Scenario05-03/", "test/Scenario05-04/"], son=True, ficher_son="mixture")
est = InertieClasificator(seuil_parole=60, bonus_parole=10)
print(est.score(data, data_ref))

[0.98442361276303036, 0.0, 0.79872782636095829, 0.78445409543082234]


Avec mixture :
res pour s = 60 et b = 10
[0.97933572191362606, 0.0, 0.78860922364016373, 0.76984948154927946]



Avec old sol
res pour s = 10 et b = 10
[0.97933572191362606, 0.0, 0.56944794123814113, 0.5517790737362408]
res pour s = 10 et b = 20
[0.97933572191362606, 0.0, 0.57959528055125398, 0.56188820954621255]
res pour s = 10 et b = 30
[0.97933572191362606, 0.0, 0.5849108605877773, 0.56716322390788698]
res pour s = 10 et b = 40
[0.97933572191362606, 0.0, 0.58675289000236186, 0.56897608476245642]
res pour s = 10 et b = 50
[0.97933572191362606, 0.0, 0.58608344814655977, 0.56832262730906513]
res pour s = 10 et b = 60
[0.97933572191362606, 0.0, 0.58471768683808933, 0.56697323965885227]
res pour s = 30 et b = 10
[0.97933572191362606, 0.0, 0.68528772330577248, 0.66706708323584296]
res pour s = 30 et b = 20
[0.97933572191362606, 0.0, 0.69795327454593747, 0.67966731346251574]
res pour s = 30 et b = 30
[0.97933572191362606, 0.0, 0.70838509416285156, 0.69002943320116839]
res pour s = 30 et b = 40
[0.97933572191362606, 0.0, 0.71116083517444284, 0.69281109704515886]
res pour s = 30 et b = 50
[0.97933572191362606, 0.0, 0.71252512392153144, 0.69419923404033235]
res pour s = 30 et b = 60
[0.97933572191362606, 0.0, 0.71427246594867977, 0.69595357981591921]
res pour s = 40 et b = 10
[0.97933572191362606, 0.0, 0.72698942912598363, 0.70852277440945199]
res pour s = 40 et b = 20
[0.97933572191362606, 0.0, 0.7406017358480006, 0.72204794188787957]
res pour s = 40 et b = 30
[0.97933572191362606, 0.0, 0.7467149548099683, 0.72813624575733626]
res pour s = 40 et b = 40
[0.97933572191362606, 0.0, 0.74877827623787918, 0.73023017425659531]
res pour s = 40 et b = 50
[0.97933572191362606, 0.0, 0.75456483660675611, 0.73602458244971425]
res pour s = 40 et b = 60
[0.97933572191362606, 0.0, 0.75087439358708163, 0.73242497022832942]
res pour s = 50 et b = 10
[0.97933572191362606, 0.0, 0.75881547943080963, 0.74015485622285471]
res pour s = 50 et b = 20
[0.97933572191362606, 0.0, 0.76236037381561927, 0.74370506980751949]
res pour s = 50 et b = 30
[0.97933572191362606, 0.0, 0.76631123963746839, 0.74766466592147185]
res pour s = 50 et b = 40
[0.97933572191362606, 0.0, 0.76915178057223899, 0.75051930029975011]
res pour s = 50 et b = 50
[0.97933572191362606, 0.0, 0.76568017277546252, 0.74713823663030732]
res pour s = 50 et b = 60
[0.97933572191362606, 0.0, 0.76221722844668771, 0.74373181502647312]
res pour s = 60 et b = 10
[0.97933572191362606, 0.0, 0.76632689197728809, 0.74768470054277669]
res pour s = 60 et b = 20
[0.97933572191362606, 0.0, 0.76966037165995926, 0.7510268796613615]
res pour s = 60 et b = 30
[0.97933572191362606, 0.0, 0.77162185607558431, 0.7530090617243882]
res pour s = 60 et b = 40
[0.97933572191362606, 0.0, 0.76606154828149209, 0.74757590553757047]
res pour s = 60 et b = 50
[0.97933572191362606, 0.0, 0.76273420979931605, 0.74430259555193534]
res pour s = 60 et b = 60
[0.97933572191362606, 0.0, 0.75859905619589008, 0.740210690706538]
res pour s = 70 et b = 10
[0.97933572191362606, 0.0, 0.77002826010079173, 0.7514505584778387]
res pour s = 70 et b = 20
[0.97933572191362606, 0.0, 0.76833588746272341, 0.74981105698064954]
res pour s = 70 et b = 30
[0.97933572191362606, 0.0, 0.76648859221204269, 0.74803684928302228]
res pour s = 70 et b = 40
[0.97933572191362606, 0.0, 0.76204489395021591, 0.74366094138372119]
res pour s = 70 et b = 50
[0.97933572191362606, 0.0, 0.75768200146947817, 0.73934517674058287]
res pour s = 70 et b = 60
[0.97933572191362606, 0.0, 0.75399964724690782, 0.73571606273651002]
res pour s = 80 et b = 10
[0.97933572191362606, 0.0, 0.77021448710370988, 0.75172487450519376]
res pour s = 80 et b = 20
[0.97933572191362606, 0.0, 0.76878370981111721, 0.7503564237996454]
res pour s = 80 et b = 30
[0.97933572191362606, 0.0, 0.76344304672966457, 0.74510906395662013]
res pour s = 80 et b = 40
[0.97933572191362606, 0.0, 0.75943038139047836, 0.74114283413417692]
res pour s = 80 et b = 50
[0.97933572191362606, 0.0, 0.75379284748373154, 0.73557427958996457]
res pour s = 80 et b = 60
[0.97933572191362606, 0.0, 0.75083849105201106, 0.73267763322755863]
res pour s = 100 et b = 10
[0.97933572191362606, 0.0, 0.73975705552957605, 0.72172468987163818]
res pour s = 100 et b = 20
[0.97933572191362606, 0.0, 0.73453728773585958, 0.7165840987865284]
res pour s = 100 et b = 30
[0.97933572191362606, 0.0, 0.72790094109587655, 0.71001509190076262]
res pour s = 100 et b = 40
[0.97933572191362606, 0.0, 0.72621807714366604, 0.70838863794838725]
res pour s = 100 et b = 50
[0.97933572191362606, 0.0, 0.72814271638911654, 0.71031363613086462]
res pour s = 100 et b = 60
[0.97933572191362606, 0.0, 0.72425450541800707, 0.70645992844335737]

In [15]:
PATH = "data/Scenario03-05/"
data_ref = read_file(PATH+"groundTruth.txt")
data = np.loadtxt(PATH+"augmented_data.txt")
data_son = np.loadtxt(PATH+"ssl_detections.txt")
est = InertieClasificator()
res = est.predict([(data, data_son)])
smax = 0
bmax = 0
max = 0
for s in [10, 30, 40, 50, 60, 70, 80, 100,110]:
    for b in [10,20,30,40,50,60]:
        est = InertieClasificator(seuil_parole=s, bonus_parole=b)
        print("res pour s = {} et b = {}".format(s, b))
        score = est.score([(data, data_son)], [data_ref])
        print(score)
        if score[3] > max:
            max = score[3]
            smax=s
            bmax = b
            
print(smax)
print(bmax)

res pour s = 10 et b = 10


[0.98476383951244284, 0.0, 0.70187713310580202, 0.68901303538175052]
res pour s = 10 et b = 20


[0.98476383951244284, 0.0, 0.70904436860068254, 0.69612324360927713]
res pour s = 10 et b = 30


[0.98476383951244284, 0.0, 0.71843003412969286, 0.70543423057389543]
res pour s = 10 et b = 40


[0.98476383951244284, 0.0, 0.72030716723549493, 0.70729642796681902]
res pour s = 10 et b = 50


...

[0.98476383951244284, 0.0, 0.69692832764505119, 0.68410360589131547]
res pour s = 110 et b = 30


[0.98476383951244284, 0.0, 0.69266211604095562, 0.67987133908921615]
res pour s = 110 et b = 40


[0.98476383951244284, 0.0, 0.68941979522184305, 0.67665481631962077]
res pour s = 110 et b = 50


[0.98476383951244284, 0.0, 0.6808873720136519, 0.66819028271542236]
res pour s = 110 et b = 60


[0.98476383951244284, 0.0, 0.66399317406143343, 0.65143050617910947]
60
30


In [19]:
import pickle

from utils import iter_frame, Corp
from utils.identity_assignation import identity_assignation


class InertieClasificator(BaseEstimator, ClassifierMixin):
    def __init__(self, coef_ine_hd = 1, coef_ine_bd =1, coef_dist_hd = 1, coef_dist_bd = 1, 
                 coef_size_hd = 1, coef_size_bd = 1, coef_pt_cache = 10, mem_length= 20, coef_col = 1,
                 seuil_parole = 60, bonus_parole = 30):
        
        args, _, _, values = inspect.getargvalues(inspect.currentframe())
        values.pop("self")
        self.seuil_parole = seuil_parole
        self.bonus_parole = bonus_parole

        for arg, val in values.items():
            setattr(self, arg, val)
            
        with open("learn/mixture_estimator") as file:
            self.est = pickle.load(file)
        with open("learn/scal") as file:
            self.scal = pickle.load(file)
    
    def fit(self, X, y=None):
        # on n'apprend rien pour l'instant
        pass
    
    def mk_dist(self):
        def fdist(id, corp):
            var = id.comp(corp)
            dist = var[0] * self.coef_ine_hd
            dist += var[1] * self.coef_ine_bd
            dist += var[2] * self.coef_col
            dist += var[3] * self.coef_dist_hd
            dist += var[4] * self.coef_dist_bd
            dist += var[5] * self.coef_pt_cache
            return dist
        return fdist
               
    
    def predict(self, X, y=None):
        # prediction
        
        def func (data):
            data_pos, data_son = data
            identites = dict()
            for frame_pos, frame_son in iter_frame_sync(data_pos,data_son):
                
                son = frame_son[0]
                identites = identity_assignation(frame_pos, identites, None, self.mk_dist())
    
                for i, l in enumerate(frame_pos):
                    cp = Corp(l)
                    id = None
                    parle = False
                    for _id in identites.keys():
                        if _id == cp:
                            id = _id
                    if frame_son[0][1] == -1:
                        parle = False
                    else:
                        cmp = compare_corp_son(frame_son, cp)
                        # print("cmp : " + str(cmp))
                        # print("mix :" + str(frame_son))
                        # print("pos :" + str(cp.hd_bary))
                        # parle = self.est.predict(np.concatenate((cmp, [int(id.get_speak())])).reshape(1,-1))
                        self.scal.transform(cmp.reshape(1, -1))
                        parle = self.est.predict(cmp.reshape(1, -1))[0]
                        # print("PRED : " + str(parle))                        
                        # print("############")                        
                    id.set_speak(parle)
                    yield (identites[cp], int(parle))  
                # if frame_pos[0][0] > 50:
                #     break
                # 
        return [make_pred(vid[0], func, data_son=vid[1]) for vid in X]
    
    def score(self, X, y, sample_weight=None):
        pred = self.predict(X)
        res = [0, 0, 0, 0]
        for vid, raw in zip(pred, y):
            c = compute_clear(raw, vid, 1)
            c = list(c)
            res = [a+b for a,b in zip(res, c)]
        # print(res/len(y))
        return [x/len(y) for x in res]

In [3]:
data_ref, data = mult_load(["data/Scenario03-02/", "data/Scenario01-01/", "data/Scenario02-03/", "data/Scenario02-01/",
                            "data/Scenario02-02/", "data/Scenario03-03/", "data/Scenario03-02/", "data/Scenario03-01/",
                            "data/Scenario03-05/"], son=True, ficher_son="mixture")

In [7]:
est = InertieClasificator()
print(est.score(data, data_ref))

IndexError: invalid index to scalar variable.

In [8]:
PATH = "data/Scenario01-01/"
data_ref = read_file(PATH+"groundTruth.txt")
data = np.loadtxt(PATH+"augmented_data.txt")
data_son = np.loadtxt(PATH+"mixture")
est = InertieClasificator()
res = est.predict([(data, data_son)], [data_ref])
print(est.score([(data, data_son)], [data_ref]))
save_data(res[0], PATH +  "res_tmp.txt")

[0.98429319371727753, 0.0, 0.84816753926701571, 0.83246073298429324]


In [14]:
from utils import compute_error
compute_error(data_ref, res, 1)

AttributeError: 'list' object has no attribute 'copy'